In [100]:
import numpy as np
import pandas as pd

In [101]:
# Load feature data
X_train = pd.read_csv('housing_X_train.csv', header=None).values.T
X_test = pd.read_csv('housing_X_test.csv', header=None).values.T

# Load target data
y_train = pd.read_csv('housing_y_train.csv', header=None).values.flatten()
y_test = pd.read_csv('housing_y_test.csv', header=None).values.flatten()

In [102]:
def mean_squared_error(y_true, y_pred):
  
    return np.mean((y_true - y_pred) ** 2)

# Ridge regression algorithm using the closed form solution for linear regression

In [103]:
def ridge_regression_closed_form(X, y, lambda_reg):
    # Number of samples and features
    n_samples, n_features = X.shape
    
    # Add intercept term by appending a column of ones to X
    X_b = np.hstack([np.ones((n_samples, 1)), X])
    
    # Create the regularization matrix
    I = np.eye(n_features + 1)
    I[0, 0] = 0  
    
    # Compute (X^T X + λI)
    XtX = X_b.T @ X_b
    XtX_plus_lambdaI = XtX + n_samples*lambda_reg * I
    
    # Compute X^T y
    Xty = X_b.T @ y
    
    # Solve for beta
    beta = np.linalg.solve(XtX_plus_lambdaI, Xty)
    
    # Extract intercept and coefficients
    intercept = beta[0]
    coefficients = beta[1:]
    
    return intercept, coefficients


In [104]:
# Define the list of lambda (λ) values to test
lambdas = [0, 0.25, 0.5, 0.75, 1]

# Initialize a list to store results
results = []

print("\nTraining and Testing Errors for Different λ Values using Ridge Regression closed-form solution:\n")
print(f"{'λ':<5} {'Train MSE':<15} {'Test MSE':<15}")
print("-" * 35)

for lambda_reg in lambdas:
    # Train the model using Ridge Regression closed-form solution
    intercept, coefficients = ridge_regression_closed_form(X_train, y_train, lambda_reg)
    
    # Make predictions on the training set
    y_train_pred = X_train @ coefficients + intercept
    
    # Make predictions on the testing set
    y_test_pred = X_test @ coefficients + intercept
    
    # Compute MSE for training and testing sets
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    
    # Store the results
    results.append({
        'Lambda': lambda_reg,
        'Train MSE': train_mse,
        'Test MSE': test_mse
    })
    
    
    print(f"{lambda_reg:<5} {train_mse:<15.2f} {test_mse:<15.2f}")


Training and Testing Errors for Different λ Values using Ridge Regression closed-form solution:

λ     Train MSE       Test MSE       
-----------------------------------
0     9.69            370.22         
0.25  15.01           65.42          
0.5   18.42           48.02          
0.75  20.42           42.22          
1     21.76           39.89          


In [105]:
# Compute mean and standard deviation from the training data
feature_means = np.mean(X_train, axis=0)
feature_stds = np.std(X_train, axis=0)

# Avoid division by zero by setting std to 1 where std is 0
feature_stds[feature_stds == 0] = 1

# Standardize training data
X_train_scaled = (X_train - feature_means) / feature_stds

# Standardize testing data using training data statistics
X_test_scaled = (X_test - feature_means) / feature_stds

# Ridge regression algorithm using the Gradient Descent

In [106]:
def ridge_regression_gradient_descent(X, y, lambda_reg, learning_rate, n_iterations):
    # Number of samples and features
    n_samples, n_features = X.shape
    
    # Initialize coefficients (including intercept)
    beta = np.zeros(n_features + 1)
    
    # Add intercept term by appending a column of ones to X
    X_b = np.hstack([np.ones((n_samples, 1)), X])
    
    # Gradient Descent
    for iteration in range(n_iterations):
        
        y_pred = X_b @ beta
        
        # Gradient of the loss function
        error = y_pred - y
        gradient = (2 / n_samples) * (X_b.T @ error) + 2 * lambda_reg * np.r_[0, beta[1:]] 
        
        # Update coefficients
        beta -= learning_rate * gradient
    
    # Extract intercept and coefficients
    intercept = beta[0]
    coefficients = beta[1:]
    
    return intercept, coefficients

In [107]:
# Set learning rate and number of iterations
learning_rate= 0.01  
n_iterations = 1000  

# Initialize a list to store results
results = []

print("\nTraining and Testing Errors for Different λ Values using Ridge Regression with gradient descent:\n")
print(f"{'λ':<5} {'Train MSE':<15} {'Test MSE':<15}")
print("-" * 35)

for lambda_reg in lambdas:
    # Train the model using Ridge Regression with gradient descent
    intercept, coefficients = ridge_regression_gradient_descent(X_train_scaled, y_train, lambda_reg, learning_rate, n_iterations)
    
    # Make predictions on the training set
    y_train_pred = X_train_scaled @ coefficients + intercept
    
    # Make predictions on the testing set
    y_test_pred = X_test_scaled @ coefficients + intercept
    
    # Compute MSE for training and testing sets
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    
    # Store the results
    results.append({
        'Lambda': lambda_reg,
        'Train MSE': train_mse,
        'Test MSE': test_mse
    })
    
    # Print the results
    print(f"{lambda_reg:<5} {train_mse:<15.2f} {test_mse:<15.2f}")


Training and Testing Errors for Different λ Values using Ridge Regression with gradient descent:

λ     Train MSE       Test MSE       
-----------------------------------
0     9.70            338.91         
0.25  11.79           85.32          
0.5   14.58           53.42          
0.75  17.38           44.47          
1     20.06           41.20          


# Lasso regression algorithm using Gradient Descent

In [108]:
def lasso_regression_gradient_descent(X, y, lambda_reg, learning_rate, n_iterations):
    # Number of samples and features
    n_samples, n_features = X.shape
    
    # Initialize coefficients (including intercept)
    beta = np.zeros(n_features + 1)
    
    # Add intercept term by appending a column of ones to X
    X_b = np.hstack([np.ones((n_samples, 1)), X])
    
    # Gradient Descent with Sub-Gradient for L1 Regularization
    for iteration in range(n_iterations):
        # Predictions: y_pred = X_b @ beta
        y_pred = X_b @ beta
        
        # Compute error
        error = y_pred - y
        
        # Gradient of the loss function (Mean Squared Error)
        gradient_loss = (2 / n_samples) * (X_b.T @ error)
        
        # Compute sub-gradient for L1 regularization
        #np.sign(0) returns 0, which is a valid sub-gradient
        sub_gradient = lambda_reg * np.sign(beta)
        sub_gradient[0] = 0  
        
        # Total gradient
        gradient = gradient_loss + sub_gradient
        
        # Update coefficients
        beta -= learning_rate * gradient
        
    # Extract intercept and coefficients
    intercept = beta[0]
    coefficients = beta[1:]
    
    return intercept, coefficients


In [109]:
# Set learning rate and number of iterations
learning_rate= 0.01  
n_iterations = 1000 

# Initialize a list to store results
results = []

print("\nTraining and Testing Errors for Different λ Values using Lasso Regression with gradient descent:\n")
print(f"{'λ':<5} {'Train MSE':<15} {'Test MSE':<15}")
print("-" * 35)

for lambda_reg in lambdas:
    # Train the model using Lasso Regression with gradient descent
    intercept, coefficients = lasso_regression_gradient_descent(X_train_scaled, y_train, lambda_reg, learning_rate, n_iterations)
    
    # Make predictions on the training set
    y_train_pred = X_train_scaled @ coefficients + intercept
    
    # Make predictions on the testing set
    y_test_pred = X_test_scaled @ coefficients + intercept
    
    # Compute MSE for training and testing sets
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    
    # Store the results
    results.append({
        'Lambda': lambda_reg,
        'Train MSE': train_mse,
        'Test MSE': test_mse
    })
    
    # Print the results
    print(f"{lambda_reg:<5} {train_mse:<15.2f} {test_mse:<15.2f}")


Training and Testing Errors for Different λ Values using Lasso Regression with gradient descent:

λ     Train MSE       Test MSE       
-----------------------------------
0     9.70            338.91         
0.25  10.10           67.10          
0.5   10.57           64.61          
0.75  11.28           64.83          
1     12.12           65.53          


In [111]:
# Ridge Gradient Descent
intercept_gd, coefficients_gd = ridge_regression_gradient_descent(X_train_scaled, y_train, lambda_reg, learning_rate, n_iterations)
#Lasso Gradient Descent
intercept_l, coefficients_l = lasso_regression_gradient_descent(X_train_scaled, y_train, lambda_reg, learning_rate, n_iterations)

print("Ridge Regression using Gradient Descent Coefficients:\n", coefficients_gd)

print("Lasso Regression using Descent Coefficients:\n", coefficients_l)


Ridge Regression using Gradient Descent Coefficients:
 [-0.01119849  0.3946084  -0.44069534  0.25901836 -0.14501965  2.91693341
 -0.31266908 -0.62279104  0.16431131 -0.64867437 -1.16269428  0.4164327
 -1.6043875 ]
Lasso Regression using Descent Coefficients:
 [-1.44517759e-03 -6.13581190e-03 -5.82747042e-03  4.27393042e-03
 -6.37042393e-03  6.32281854e+00 -4.16889426e-02 -6.47756942e-03
  3.50778507e-03 -4.95216935e-01 -1.09186763e+00  9.91342616e-02
 -8.50224303e-01]
